In [26]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Procura o que retorna a primeira página do Google, a partir de buscas feitas com o conteúdo de um dataframe e uma palavra-chave

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support.select import Select
import pandas as pd
from bs4 import BeautifulSoup
import time

In [4]:
candidatos = pd.read_excel('/home/reinaldo/Documentos/Code/ti/repo/besouro/resultados/consolidado/planilha_selecionados_dia3set.xlsx',sheet_name='Planilha1', converters={'cpf': lambda x: str(x),
'numero_urna': lambda x: str(x)})

In [5]:
candidatos = candidatos[(candidatos['pacto'] == 'SEM RESPOSTA')]

In [6]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 234
Data columns (total 32 columns):
uf                              162 non-null object
cargo                           162 non-null object
numero_urna                     162 non-null object
nome_completo                   162 non-null object
nome_urna                       162 non-null object
cpf                             162 non-null object
e_mail_tse                      162 non-null object
partido_eleicao                 162 non-null object
data_nascimento                 162 non-null object
ocupacao_tse                    162 non-null object
total_bens                      162 non-null object
detalhes_bens                   162 non-null object
total_inqueritos                162 non-null object
total_acoes_penais              162 non-null object
detalhes_processos              162 non-null object
reu_no_stf                      162 non-null object
e_mail_survey                   162 non-null object
ocupacao_su

In [31]:
lista = []

In [33]:
for num, row in candidatos.iterrows():
    cpf = (row['cpf']).strip()
    politico = (row['nome_urna']).strip()
    print(politico)
    cargo = (row['cargo']).strip()
    partido = (row['partido_eleicao']).strip()
    
    nome = politico.replace(' ', '+')
    nome = '"'+ nome + '"'
        
    busca = f'https://www.google.com.br/search?q={nome}+{cargo}+{partido}+ditadura'
        
    profile = webdriver.FirefoxProfile()
    browser = webdriver.Firefox(profile)
    browser.implicitly_wait(10)
    
    browser.get(busca)
    
    time.sleep(10)
    
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")
    browser.close()
    
    page = soup.find_all("div", {"class": "rc"})
    
    for link in page:
        href = link.find("a")['href']
        nome_pagina = link.find("a").text
        body = link.find('span', attrs={'class': 'st'}).text
        lista.append({'cpf': cpf, 
                      'politico': politico, 
                      'partido_eleicao': partido, 
                      'link': href,
                      'nome_pagina': nome_pagina,
                      'texto_resumo': body
                     })

ALDEANNO CAMPOS
ISABEL TEIXEIRA
ALEXANDRE MACHADO
FABIO CHAZYN
RAUL THAME
EDSON ALMEIDA
MACHADO
IRIA CABRERA
MARCONI
MARCELO VIANA
MAURO MATTIA
MARISA LOBO PSICOLOGA CRISTÃ
FRANCISCO LUCIO
SEU GABINETE
MARCUS VICENTE
INACIO HENRIQUE
DR. RICARDO SMARZARO
AGENTE ELIAS
DEMONTIE
CARLOS BETTIATI
FABIO MACCA
ADÉRMIS MARINI
LUIZ CLAUDIO ROSA
CAIO SAMPAIO
PAULO GANIME
ATAÍDES OLIVEIRA
EDUARDO CUSTO ZERO
FELIPE RIGONI
FRANCISCO FERNANDES
ADRIANA DORNELLES
LEO MESQUITA
LUCAS GONZALEZ
CAROLA  GRANATO
CRISTOVAM
ROSIDALVA AQUINO
DÉBORA
CABO GLAUCIA
DAGOSTINO
CARLA VITÓRIA
SARGENTO MARTINS
DURVAL PARREIRA
MONICA ROSENBERG
EDSON SARDANO
WICK MANFRINATO
FERRAZ JUNIOR
JULIANA A. CARDOSO
MAURI DA MOTA
FERNANDO MATRIX
BRUNO EIZERIK
DARIA CRISTINA
RODRIGO BORGES
ADOLFO FURTADO
COMANDANTE SARMENTO
JUAN CARLOS
MISAEL NEMESEK
ROLAND HASSON
TIAGO MARTINES
JACKSON CABANAS
WALTER STANULA
CORONEL ROCHA
JARDEL BARROS
CAIO CHECON
CEZAR ALMEIDA
CORONEL RAMALHO
ASSIS MARINHO
ODECIO CARNEIRO
JORGE FONTENELLE
CORONEL 

In [41]:
df_tela = pd.DataFrame(lista, columns = ['cpf', 'link', 'nome_pagina', 'partido_eleicao', 'politico', 'texto_resumo'])

In [42]:
df_tela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622 entries, 0 to 621
Data columns (total 6 columns):
cpf                622 non-null object
link               622 non-null object
nome_pagina        622 non-null object
partido_eleicao    622 non-null object
politico           622 non-null object
texto_resumo       622 non-null object
dtypes: object(6)
memory usage: 29.2+ KB


In [44]:
df_tela.reset_index().head()

,index,cpf,link,nome_pagina,partido_eleicao,politico,texto_resumo
0,0,59826690244,https://stackoverflow.com/questions/52157077/w...,"selenium webdriver - With Python, how to scrap...",PRP,ALDEANNO CAMPOS,17 horas atrás - ... f'https://www.google.com....
1,1,59826690244,https://stackoverflow.com/questions/52157077/w...,"selenium webdriver - With Python, how to scrap...",PRP,ALDEANNO CAMPOS,9 horas atrás - ... cargo = 'DEPUTADO FEDERAL'...
2,2,59826690244,https://iphone.facebook.com/oeducabral/?__tn__...,Eduardo Cabral - Home | Facebook,PRP,ALDEANNO CAMPOS,ALDEANNO CAMPOS added 14 new photos. February ...
3,3,59826690244,https://touch.facebook.com/JesusCristoCaminhoq...,Atualidade e Cristianismo - Home | Facebook,PRP,ALDEANNO CAMPOS,ALDEANNO CAMPOS added 14 new photos. February ...
4,4,08764240860,https://twitter.com/BolsonaroSP/status/9776806...,"Eduardo Bolsonaro on Twitter: ""Breve aula de h...",NOVO,ISABEL TEIXEIRA,"Filho do deputado federal Jair Bolsonaro, Poli..."


In [45]:
df_tela.to_excel('buscagoogle_candidatos_ditadura.xlsx',sheet_name='Sheet1')